In [18]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [19]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
29,This is what I was expecting when star trek DS...,positive
160,The Salena Incident is set in Arizona where si...,negative
590,"Twenty five years ago, I showed this film in s...",positive
601,I suppose if you like pure action... you'll fi...,negative
376,I found the pace to be glacial and the origina...,negative


In [20]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [7]:
# import nltk
# nltk.download()

In [8]:
df = normalize_text(df)
df.head()

,review,sentiment
71,hugh ed harris hotshot bachelor senator determ...,positive
775,labor love frame picture perfect grab you shee...,positive
416,now like sci fi cartoon however robotboy appea...,negative
283,and episode really hoping not mention canceled...,negative
856,ann margret best job ever done history film ma...,positive


In [9]:
df['sentiment'].value_counts()

sentiment
negative    251
positive    249
Name: count, dtype: int64

In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [11]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
71,hugh ed harris hotshot bachelor senator determ...,1
775,labor love frame picture perfect grab you shee...,1
416,now like sci fi cartoon however robotboy appea...,0
283,and episode really hoping not mention canceled...,0
856,ann margret best job ever done history film ma...,1


In [12]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Sonalikasingh17/Capstone-Project.mlflow')
dagshub.init(repo_owner='Sonalikasingh17', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as Sonalikasingh17

Initialized MLflow to track repo "Sonalikasingh17/Capstone-Project"

Repository Sonalikasingh17/Capstone-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/86908e2b6fa0407b9cf71036aac76a22', creation_time=1760072677372, experiment_id='0', last_update_time=1760072677372, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [17]:
import mlflow
import logging
import os
import time
import joblib
import nbformat
from nbformat.v4 import new_notebook, new_code_cell
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ==============================
# Configure logging
# ==============================
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

# ==============================
# Start MLflow Run
# ==============================

with mlflow.start_run():
    start_time = time.time()
    
    try:
        # ==============================
        # Data Loading & Preprocessing
        # ==============================

        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        # ==============================
        # Model Training
        # ==============================

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")


        # ==============================
        # Model Evaluation
        # ==============================

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        # ==============================
        # Save & Log Model
        # ==============================

        # Save model locally in pickle format
        # mlflow.sklearn.log_model(model, "model")
        logging.info("Saving and logging the model...")
        model_path = os.path.abspath("logistic_regression_model.pkl")
        joblib.dump(model, model_path)

        # Log it as an artifact
        mlflow.log_artifact(model_path)
        logging.info("Model saved and logged successfully.")
    
        # ==============================
        # Log execution time
        # ==============================
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        notebook_path = "exp1.ipynb"
        logging.info("Executing Jupyter Notebook. This may take a while...")
        os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        mlflow.log_artifact(notebook_path)

        logging.info("Notebook execution and logging complete.")

        
        # # ============================== 
        # # Create & Log Notebook
        # # ==============================
        # notebook_path = "exp1_baseline_model.ipynb"
        # abs_notebook_path = os.path.abspath(notebook_path)

        # if not os.path.exists(abs_notebook_path):
        #     logging.info(f"Notebook '{notebook_path}' not found. Creating a new one...")
        #     nb = new_notebook(
        #         cells=[
        #             new_code_cell(
        #                 """# Experiment 1: Baseline Logistic Regression Model
        #                                     import joblib
        #                                     print("Notebook auto-generated during MLflow run.")
        #                                     model = joblib.load("logistic_regression_model.pkl")
        #                                     print("Loaded model:", model)"""
        #             )
        #         ]
        #     )
        #     with open(abs_notebook_path, "w", encoding="utf-8") as f:
        #         nbformat.write(nb, f)
        #     logging.info("Notebook created successfully.")

        # # Execute and log the notebook
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {abs_notebook_path}")
        # mlflow.log_artifact(abs_notebook_path)
        # logging.info("Notebook execution and logging complete.")

        # ==============================
        # Print Metrics
        # ==============================
    
        logging.info(f"Accuracy: {accuracy:.4f}")
        logging.info(f"Precision: {precision:.4f}")
        logging.info(f"Recall: {recall:.4f}")
        logging.info(f"F1 Score: {f1:.4f}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-10-10 11:38:09,954 - INFO - Starting MLflow run...


2025-10-10 11:38:11,762 - INFO - Logging preprocessing parameters...
2025-10-10 11:38:12,850 - INFO - Initializing Logistic Regression model...
2025-10-10 11:38:12,850 - INFO - Fitting the model...
2025-10-10 11:38:12,879 - INFO - Model training complete.
2025-10-10 11:38:12,879 - INFO - Logging model parameters...
2025-10-10 11:38:13,247 - INFO - Making predictions...
2025-10-10 11:38:13,247 - INFO - Calculating evaluation metrics...
2025-10-10 11:38:13,256 - INFO - Logging evaluation metrics...
2025-10-10 11:38:17,762 - INFO - Saving and logging the model...
2025-10-10 11:38:19,891 - INFO - Model saved and logged successfully.
2025-10-10 11:38:19,891 - INFO - Model training and logging completed in 8.13 seconds.
2025-10-10 11:38:19,891 - INFO - Executing Jupyter Notebook. This may take a while...
2025-10-10 11:38:21,223 - INFO - Notebook execution and logging complete.
2025-10-10 11:38:21,227 - INFO - Accuracy: 0.7100
2025-10-10 11:38:21,227 - INFO - Precision: 0.7500
2025-10-10 11:3

🏃 View run useful-dog-108 at: https://dagshub.com/Sonalikasingh17/Capstone-Project.mlflow/#/experiments/0/runs/bdc70487e5034fd28162505a4382a849
🧪 View experiment at: https://dagshub.com/Sonalikasingh17/Capstone-Project.mlflow/#/experiments/0
